In [1]:
from glob import glob
from os import path
import seaborn as sns
import json
import pandas as pd
import os
import numpy as np
import re
from os.path import basename, splitext
import time


/home/clarice/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
list_city_dt = []

for file_in in os.listdir(r'../../instances'):
    base_file = splitext(basename(file_in))[0]
    base_file_split = base_file.split('_')
    file = base_file_split[0]+'_'+base_file_split[1]
    list_city_dt.append(file)

list_city_dt = list(set(list_city_dt))
list_city_dt.sort()

#remove berlin
val_rmv = []
for val in list_city_dt:
    if val.find('berlin') > -1:
        val_rmv.append(val)
for val in val_rmv:
    list_city_dt.remove(val)

print(list_city_dt)
print(len(list_city_dt))


['frankfurt_db=0.50', 'frankfurt_db=1.00', 'frankfurt_db=2.00', 'frankfurt_db=4.00', 'lyon_db=0.50', 'lyon_db=1.00', 'lyon_db=2.00', 'lyon_db=4.00', 'paris_db=0.50', 'paris_db=1.00', 'paris_db=2.00', 'paris_db=4.00']
12


In [3]:

list_workforce_xlsx = []

for base_file in list_city_dt:

    base_file_xlsx = f'{base_file}_all_trials.xlsx'
    if base_file_xlsx in os.listdir(r'../../rerun_workforce_size'):
        df_ = pd.read_excel(f'../../rerun_workforce_size/{base_file_xlsx}')
        list_workforce_xlsx.append(df_)


In [11]:
df_workforce = pd.concat(list_workforce_xlsx, ignore_index = True)

df_workforce.columns = ['instance', 'city', 'DB', 'OC',
       'RM', 'GM', 'model', 'max_n_shifts',
       'workforce_size', 'wage_costs', 'objective_value',
       'objective_value_post_wage', 'objval_round','workforce_size_trial','run_time','dict_run']
df_workforce.loc[df_workforce['max_n_shifts'].isna(), 'max_n_shifts'] = 0
df_workforce.drop_duplicates(subset = ['instance','OC','model','max_n_shifts','workforce_size_trial'], inplace = True)
# df_workforce['max_'] = df_workforce.groupby(by = ['instance','OC','model','max_n_shifts'])['workforce_size_trial'].transform('max')
# df_workforce = df_workforce[df_workforce['max_']!=df_workforce['workforce_size_trial']]

# df_workforce['max_'] = df_workforce.groupby(by = ['instance','OC','model','max_n_shifts'])['workforce_size_trial'].transform('max')
# df_workforce = df_workforce[df_workforce['max_']==df_workforce['workforce_size_trial']]
# df_workforce.drop(columns = ['max_'], inplace = True)

df_workforce.head(2)

,instance,city,DB,OC,RM,GM,model,max_n_shifts,workforce_size,wage_costs,objective_value,objective_value_post_wage,objval_round,workforce_size_trial,run_time,dict_run
0,frankfurt_db=0.50,frankfurt,0.5,1.2,1.5,0.8,fixed,0.0,13,312.0,1918.68,1606.68,1918.68,10,2.399602,"{'fixed-None': [], 'flex-None': ['partflex-3',..."
1,frankfurt_db=0.50,frankfurt,0.5,1.2,1.5,0.8,fixed,0.0,14,336.0,1913.32,1577.32,1913.32,11,2.702752,"{'fixed-None': [], 'flex-None': ['partflex-3',..."


In [7]:
df_workforce.loc[0,'objective_value']

1918.680000000052

In [84]:
#check to see if the objective value says consistent for more than two iterations

df_workforce.sort_values(by = ['instance','OC','model','max_n_shifts','workforce_size_trial'], ascending = [True, True, True, True, False], inplace = True)

df_workforce['lag_1'] = df_workforce.groupby(by = ['instance','OC','model','max_n_shifts'])['objective_value'].shift(1)
df_workforce['lag_2'] = df_workforce.groupby(by = ['instance','OC','model','max_n_shifts'])['objective_value'].shift(2)
df_workforce['lag_3'] = df_workforce.groupby(by = ['instance','OC','model','max_n_shifts'])['objective_value'].shift(3)
df_workforce['lag_4'] = df_workforce.groupby(by = ['instance','OC','model','max_n_shifts'])['objective_value'].shift(4)

df_workforce['diff_is_zero'] = 0
for col in ['lag_1','lag_2','lag_3','lag_4']:
    df_workforce[col+'_diff'] = (df_workforce['objective_value']-df_workforce[col]).round(8)
    df_workforce.loc[df_workforce[col+'_diff']==0, 'diff_is_zero'] = 1

df_workforce.head(5)

df_workforce.to_excel(r'../code_cm/all_workforces_only_optimal.xlsx', index = False)


In [89]:
#places where lag_2_diff is zero

df_check = df_workforce[df_workforce['lag_2_diff']==0].copy()
print(len(df_check['instance']))
df_check.head(5)

9


,instance,city,DB,OC,RM,GM,model,max_n_shifts,workforce_size,wage_costs,...,workforce_size_trial,lag_1,lag_2,lag_3,lag_4,diff_is_zero,lag_1_diff,lag_2_diff,lag_3_diff,lag_4_diff
337,frankfurt_db=1.00,frankfurt,1.0,1.2,1.5,0.8,fixed,0.0,24,568.0,...,21,3681.853333,3681.853333,NaN,NaN,1,0.0,0.0,NaN,NaN
362,frankfurt_db=1.00,frankfurt,1.0,1.2,1.5,0.8,partflex,2.0,24,568.0,...,21,3681.853333,3681.853333,NaN,NaN,1,0.0,0.0,NaN,NaN
542,frankfurt_db=1.00,frankfurt,1.0,2.0,1.5,0.8,flex,0.0,24,576.0,...,21,5673.777778,5673.777778,NaN,NaN,1,0.0,0.0,NaN,NaN
571,frankfurt_db=1.00,frankfurt,1.0,2.0,1.5,0.8,partflex,3.0,24,576.0,...,21,5673.777778,5673.777778,NaN,NaN,1,0.0,0.0,NaN,NaN
586,frankfurt_db=1.00,frankfurt,1.0,2.0,1.5,0.8,partflex,4.0,24,576.0,...,21,5673.777778,5673.777778,NaN,NaN,1,0.0,0.0,NaN,NaN


In [12]:
df_workforce['count'] = 1
df_workforce['count'] = df_workforce.groupby(by = ['instance','OC','model','max_n_shifts'])['count'].transform('sum')

df_workforce_count = df_workforce.copy()
df_workforce_count.drop_duplicates(subset = ['instance','OC','model','max_n_shifts'], inplace = True)

df_workforce_count.sort_values(by = ['count'], inplace = True)

In [14]:
df_workforce_count_ = df_workforce_count[df_workforce_count['count']==3].copy()
df_workforce_count_

,instance,city,DB,OC,RM,GM,model,max_n_shifts,workforce_size,wage_costs,objective_value,objective_value_post_wage,objval_round,workforce_size_trial,run_time,dict_run,count
1278,lyon_db=4.00,lyon,4.0,2.5,1.5,0.8,flex,0.0,82,1968.0,47995.571136,46027.571136,47995.57,79,50.494599,"{'fixed-None': ['partflex-2'], 'flex-None': ['...",3
1279,lyon_db=4.00,lyon,4.0,2.5,1.5,0.8,partflex,3.0,82,1968.0,47995.571136,46027.571136,47995.57,79,50.494599,"{'fixed-None': ['partflex-2'], 'flex-None': ['...",3
1280,lyon_db=4.00,lyon,4.0,2.5,1.5,0.8,partflex,4.0,82,1968.0,47995.571136,46027.571136,47995.57,79,50.494599,"{'fixed-None': ['partflex-2'], 'flex-None': ['...",3


In [25]:
df_data = df_workforce.copy()

df_data= df_data[(df_data['city']=='berlin')&(df_data['DB']==4)&(df_data['OC']==2.5)&(df_data['model']=='flex')]
df_data

,instance,city,DB,OC,RM,GM,model,max_n_shifts,workforce_size,wage_costs,objective_value,objective_value_post_wage,workforce_size_trial,count
2051,berlin_db=4.00,berlin,4.0,2.5,1.5,0.8,flex,0.0,4,96.0,66566.417493,66470.417493,1,44
2052,berlin_db=4.00,berlin,4.0,2.5,1.5,0.8,flex,0.0,5,120.0,66313.226852,66193.226852,2,44
2053,berlin_db=4.00,berlin,4.0,2.5,1.5,0.8,flex,0.0,6,144.0,66060.323710,65916.323710,3,44
2054,berlin_db=4.00,berlin,4.0,2.5,1.5,0.8,flex,0.0,7,168.0,65808.108069,65640.108069,4,44
2055,berlin_db=4.00,berlin,4.0,2.5,1.5,0.8,flex,0.0,8,192.0,65555.942427,65363.942427,5,44
2056,berlin_db=4.00,berlin,4.0,2.5,1.5,0.8,flex,0.0,9,216.0,65305.250661,65089.250661,6,44
2057,berlin_db=4.00,berlin,4.0,2.5,1.5,0.8,flex,0.0,10,240.0,65055.710284,64815.710284,7,44
2058,berlin_db=4.00,berlin,4.0,2.5,1.5,0.8,flex,0.0,11,264.0,64806.409987,64542.409987,8,44
2059,berlin_db=4.00,berlin,4.0,2.5,1.5,0.8,flex,0.0,12,288.0,64558.425463,64270.425463,9,44
2060,berlin_db=4.00,berlin,4.0,2.5,1.5,0.8,flex,0.0,13,312.0,64311.197884,63999.197884,10,44


In [27]:
df_workforce_count_ = df_workforce_count[df_workforce_count['count']==2]
df_workforce_count_

,instance,city,DB,OC,RM,GM,model,max_n_shifts,workforce_size,wage_costs,objective_value,objective_value_post_wage,workforce_size_trial,count


In [16]:
#check frankfurt

list_city_dt_subset = []

for city_db in list_city_dt:
    if (city_db.find('frank') > -1):
        list_city_dt_subset.append(city_db)

print(list_city_dt_subset)

['frankfurt_db=0.50', 'frankfurt_db=1.00', 'frankfurt_db=2.00', 'frankfurt_db=4.00']


In [17]:
list_frank = []

for base_file in list_city_dt_subset:
    with open(f'../../workforce_size/{base_file}.json') as f:
        data = json.load(f)
        df_ = pd.DataFrame(data)
        list_frank.append(df_)

df_frank = pd.concat(list_frank, ignore_index = True)



In [19]:
df_frank
df_frank.to_excel(r'../code_cm/check_frank_optimal.xlsx', index = False)

In [24]:
list_city_dt_subset = []

for city_db in list_city_dt:
    if (city_db.find('berlin') > -1):
        list_city_dt_subset.append(city_db)

print(list_city_dt_subset)

['berlin_db=0.50', 'berlin_db=1.00', 'berlin_db=2.00', 'berlin_db=4.00']


In [25]:
list_ = []

for base_file in list_city_dt_subset:
    with open(f'../../workforce_size/{base_file}.json') as f:
        data = json.load(f)
        df_ = pd.DataFrame(data)
        list_.append(df_)

df_all = pd.concat(list_, ignore_index = True)

df_all.to_excel(r'../code_cm/check_berlin_optimal.xlsx', index = False)
df_all

,instance_file_base,city,demand_baseline,model,max_n_shifts,outsourcing_cost_multiplier,regional_multiplier,global_multiplier,workforce_size_region0,objective_value_prev,objective_value,objective_value_next
0,berlin_db=0.50,berlin,0.5,fixed,NaN,1.2,1.5,0.8,23,NaN,3.269880e+03,3269.880000
1,berlin_db=0.50,berlin,0.5,flex,NaN,1.2,1.5,0.8,21,NaN,3.149780e+03,3149.780000
2,berlin_db=0.50,berlin,0.5,partflex,2.0,1.2,1.5,0.8,20,NaN,3.150220e+03,3150.220000
3,berlin_db=0.50,berlin,0.5,partflex,3.0,1.2,1.5,0.8,20,NaN,3.149980e+03,3149.980000
4,berlin_db=0.50,berlin,0.5,partflex,4.0,1.2,1.5,0.8,21,NaN,3.149780e+03,3149.780000
...,...,...,...,...,...,...,...,...,...,...,...,...
95,berlin_db=4.00,berlin,4.0,fixed,NaN,2.5,1.5,0.8,115,NaN,5.333186e+04,53331.858995
96,berlin_db=4.00,berlin,4.0,flex,NaN,2.5,1.5,0.8,70,NaN,1.000000e+07,NaN
97,berlin_db=4.00,berlin,4.0,partflex,2.0,2.5,1.5,0.8,115,NaN,5.333186e+04,53331.858995
98,berlin_db=4.00,berlin,4.0,partflex,3.0,2.5,1.5,0.8,70,NaN,1.000000e+07,NaN
